<a href="https://colab.research.google.com/github/Navaja970927/Tesis/blob/master/MTR_EA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# check scikit-learn version
import sklearn
print(sklearn.__version__)

1.0.2


In [ ]:
# example of multioutput regression test problem
from sklearn.datasets import make_regression
# create datasets
X, y = make_regression(n_samples=500, n_features=10, n_informative=5, n_targets=5, random_state=1, noise=0.5)
# summarize dataset
print(X.shape, y.shape)

(500, 10) (500, 5)


In [ ]:
!pip install amorf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 722 kB 8.3 MB/s 
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=dc63a22ea5ca9324372b5b898b6b99770f89c9b13e45c3e5349e07cc274da6e4
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built liac-arff


In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor
from amorf.metrics import average_relative_root_mean_squared_error
import warnings
warnings.filterwarnings('ignore')
def build_model_cv(X,y,model_type):
  if model_type=='LR':
    model = LinearRegression()
  elif model_type=='KNN':
    model = KNeighborsRegressor(algorithm='auto', 
                     leaf_size=30, 
                     metric='minkowski',
                     metric_params=None, 
                     n_jobs=1, 
                     n_neighbors=10, # default is 5
                     p=2,         # p=2 is equivalent to euclidian distance
                     weights='uniform')
  elif model_type=='DT':
    model = DecisionTreeRegressor()
  elif model_type=='RF':
    model = RandomForestRegressor(max_depth=2, random_state=0)
  elif model_type=='MRF':
    model = MultiOutputRegressor(RandomForestRegressor(max_depth=2, random_state=0))
  elif model_type=='SVR':
    model = MultiOutputRegressor(LinearSVR())
  elif model_type=='RC_SVR':
    model = RegressorChain(LinearSVR())
  elif model_type=='GB':
    model = MultiOutputRegressor(GradientBoostingRegressor())
  else:
    print('Unsuported classifier')
    return 0
  # fit model
  results=[]
  cv = RepeatedKFold(n_splits=5,n_repeats=1, random_state=25)
  for train_ix, test_ix in cv.split(X):
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse=average_relative_root_mean_squared_error(y_pred,y_test)
    
    results.append(mse)
    """
    mse=[]
    for i in range(y_test.shape[1]):
      mse.append(np.sqrt(mean_squared_error(y_test[:,i], y_pred[:,i])))
		# store result
    #print('>%.3f' % mse)
    results.append(mse)
  #n_scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
  """
  print('MAE_%s: %.3f (%.3f)' % (model_type,mean(results), std(results)))
  return results

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import RegressorChain
import numpy as np
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')
def build_model(X,y,model_type):
  if model_type=='LR':
    model = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
  elif model_type=='KNN':
    model = KNeighborsRegressor(algorithm='auto', 
                     leaf_size=30, 
                     metric='minkowski',
                     metric_params=None, 
                     n_jobs=1, 
                     n_neighbors=10, # default is 5
                     p=2,         # p=2 is equivalent to euclidian distance
                     weights='uniform')
  elif model_type=='DT':
    model = DecisionTreeRegressor()
  elif model_type=='RF':
    model = RandomForestRegressor(max_depth=2, random_state=0)
  elif model_type=='MRF':
    model = MultiOutputRegressor(RandomForestRegressor(max_depth=2, random_state=0))
  elif model_type=='SVR':
    model = MultiOutputRegressor(LinearSVR())
  elif model_type=='RC_SVR':
    model = RegressorChain(LinearSVR())
  elif model_type=='GB':
    model = MultiOutputRegressor(GradientBoostingRegressor())
  else:
    print('Unsuported classifier')
    return 0
  # fit model
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=33)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mse=[]
  for i in range(y_test.shape[1]):
    mse.append(np.sqrt(mean_squared_error(y_test[:,i], y_pred[:,i])))
  print('MSE_%s: %.3f (%.3f)' % (model_type,mean(mse), std(mse)))

  return mse 

In [ ]:
model_type=['LR','KNN','DT','RF','MRF', 'SVR','RC_SVR','GB']
result={}
result.fromkeys(model_type)
for m in model_type:
  result[m]=build_model_cv(X,y,m)
print(result)
  


MAE_LR: 0.004 (0.000)
MAE_KNN: 0.500 (0.023)
MAE_DT: 0.651 (0.061)
MAE_RF: 0.717 (0.032)
MAE_MRF: 0.616 (0.029)
MAE_SVR: 0.005 (0.000)
MAE_RC_SVR: 0.013 (0.005)
MAE_GB: 0.225 (0.011)
{'LR': [0.003786468342818941, 0.004393637989072703, 0.00469948499525428, 0.004232166690487738, 0.004526492911039851], 'KNN': [0.5276813283737234, 0.47098008570555006, 0.5045811696060591, 0.4759139146160364, 0.5223252193615726], 'DT': [0.6322152921289985, 0.7204193171913255, 0.6600507004972795, 0.5460994592350199, 0.6967490350216237], 'RF': [0.6957683866739864, 0.7156740471236134, 0.7136568093841806, 0.6834374513751804, 0.7764253358950143], 'MRF': [0.6091059308274195, 0.6112931175942335, 0.608233451522593, 0.5822356652318479, 0.6689124774591622], 'SVR': [0.004243489251077004, 0.004705743557682796, 0.00504118814980991, 0.004675202848463235, 0.004839409323235529], 'RC_SVR': [0.008800396088765743, 0.009163711001003667, 0.00832083803749843, 0.015193888946991332, 0.02148926199823476], 'GB': [0.23780211882599392,

In [ ]:
# k-nearest neighbors for multioutput regression
from sklearn.datasets import make_regression
from sklearn.neighbors import KNeighborsRegressor
# create datasets
#X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define model
model = KNeighborsRegressor()
# fit model
model.fit(X, y)
# make a prediction
#row = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
yhat1 = model.predict([row])
# summarize prediction
print(yhat1[0])

NameError: ignored

In [ ]:
# decision tree for multioutput regression
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
# create datasets
#X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define model
model = DecisionTreeRegressor()
# fit model
model.fit(X, y)
# make a prediction
#row = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
yhat2 = model.predict([row])
# summarize prediction
print(yhat2[0])